# Hugging Face Autoformer with EBM Explanation for Multivariate Forecasting

This notebook implements:
1. Data loading and preprocessing for concatenated interpolated data
2. Hugging Face Autoformer for multivariate time series forecasting
3. Model training and evaluation
4. EBM (Explainable Boosting Machine) for blend effects explanation


## 1. Setup and Dependencies


In [ ]:
# Install required packages
!pip install transformers datasets evaluate accelerate
!pip install interpret pandas numpy scikit-learn matplotlib seaborn
!pip install torch torchvision torchaudio


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoformerConfig, AutoformerForPrediction, Trainer, TrainingArguments
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from interpret.glassbox import ExplainableBoostingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages imported successfully")


## 2. Data Loading and Preprocessing


In [ ]:
# Load the concatenated interpolated data
file_path = 'final_concatenated_data_mice_imputed.csv'
df = pd.read_csv(file_path)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Define target columns (multivariate)
target_cols = [
    'Total_Stabilized_Naphtha_Product_Flowrate',
    'Total_Kerosene_Product_Flowrate', 
    'Jet_Fuel_Product_Train1_Flowrate',
    'Total_Light_Diesel_Product_Flowrate',
    'Total_Heavy_Diesel_Product_Flowrate',
    'Total_Atmospheric_Residue_Flowrate',
    'Blend_Yield_Gas & LPG',
    'Blend_Yield_Kerosene',
    'Blend_Yield_Light Diesel',
    'Blend_Yield_Heavy Diesel',
    'Blend_Yield_RCO'
]

# Define static features (blend characteristics)
static_cols = [col for col in df.columns if col.startswith('crude_') or col in ['API', 'Sulphur', 'blend_id']]

# Define time series features (excluding targets and statics)
ts_cols = [col for col in df.columns if col not in ['date'] + target_cols + static_cols]

print(f"Target columns: {len(target_cols)}")
print(f"Static columns: {len(static_cols)}")
print(f"Time series columns: {len(ts_cols)}")


In [ ]:
# Data preprocessing
def preprocess_data(df, target_cols, ts_cols, static_cols):
    # Convert date column
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    # Convert all columns to numeric
    for col in target_cols + ts_cols + static_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Handle missing values
    df = df.fillna(method='ffill').fillna(method='bfill')
    
    # Remove any remaining NaN rows
    df = df.dropna()
    
    return df

# Preprocess the data
df_processed = preprocess_data(df, target_cols, ts_cols, static_cols)
print(f"Processed dataset shape: {df_processed.shape}")
print(f"Date range: {df_processed['date'].min()} to {df_processed['date'].max()}")


## 3. Custom Dataset Class for Time Series


In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, target_cols, ts_cols, static_cols, seq_len=60, pred_len=7):
        self.data = data
        self.target_cols = target_cols
        self.ts_cols = ts_cols
        self.static_cols = static_cols
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        # Prepare features and targets
        self.features = data[ts_cols].values
        self.targets = data[target_cols].values
        self.statics = data[static_cols].values
        
        # Normalize features
        self.feature_scaler = StandardScaler()
        self.target_scaler = StandardScaler()
        
        self.features_scaled = self.feature_scaler.fit_transform(self.features)
        self.targets_scaled = self.target_scaler.fit_transform(self.targets)
        
        # Create sequences
        self.sequences = []
        self.labels = []
        self.static_features = []
        
        for i in range(len(self.features_scaled) - seq_len - pred_len + 1):
            seq = self.features_scaled[i:i+seq_len]
            label = self.targets_scaled[i+seq_len:i+seq_len+pred_len]
            static = self.statics[i+seq_len]  # Use static features at prediction time
            
            self.sequences.append(seq)
            self.labels.append(label)
            self.static_features.append(static)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return {
            'past_values': torch.FloatTensor(self.sequences[idx]),
            'past_time_features': torch.zeros(self.seq_len, 1),  # Dummy time features
            'static_categorical_features': torch.zeros(len(self.static_cols)),  # Dummy categorical
            'static_real_features': torch.FloatTensor(self.static_features[idx]),
            'future_values': torch.FloatTensor(self.labels[idx]),
            'future_time_features': torch.zeros(self.pred_len, 1)  # Dummy time features
        }
    
    def inverse_transform_targets(self, scaled_targets):
        return self.target_scaler.inverse_transform(scaled_targets)

print("✅ TimeSeriesDataset class defined")


## 4. Model Configuration and Training


In [ ]:
# Create dataset
dataset = TimeSeriesDataset(df_processed, target_cols, ts_cols, static_cols, seq_len=60, pred_len=7)
print(f"Dataset size: {len(dataset)}")

# Split data
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset = torch.utils.data.Subset(dataset, range(train_size))
val_dataset = torch.utils.data.Subset(dataset, range(train_size, train_size + val_size))
test_dataset = torch.utils.data.Subset(dataset, range(train_size + val_size, len(dataset)))

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


In [ ]:
# Configure Autoformer
config = AutoformerConfig(
    context_length=60,
    prediction_length=7,
    num_time_features=1,
    num_static_categorical_features=0,
    num_static_real_features=len(static_cols),
    cardinality=[],
    embedding_dimension=[],
    d_model=64,
    encoder_attention_heads=8,
    decoder_attention_heads=8,
    encoder_layers=2,
    decoder_layers=2,
    encoder_ffn_dim=128,
    decoder_ffn_dim=128,
    activation_function="gelu",
    dropout=0.1,
    encoder_layerdrop=0.1,
    decoder_layerdrop=0.1,
    use_cache=True,
    num_parallel_samples=100,
    init_std=0.02,
    num_patches=16,
    patch_size="auto",
    patch_stride="auto",
    num_default_real_val_embeddings=1,
    scaling="std",
    embedding_dimension_multiplier=2.0,
    distr_output="student_t",
    loss="nll",
    input_size=len(ts_cols),
    num_targets=len(target_cols),
    output_size=len(target_cols),
)

print("✅ Autoformer configuration created")


In [ ]:
# Create model and training setup
model = AutoformerForPrediction(config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./autoformer_results",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Custom compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Calculate metrics for each target
    mse = mean_squared_error(labels.flatten(), predictions.flatten())
    mae = mean_absolute_error(labels.flatten(), predictions.flatten())
    r2 = r2_score(labels.flatten(), predictions.flatten())
    
    return {
        "mse": mse,
        "mae": mae,
        "r2": r2,
        "rmse": np.sqrt(mse)
    }

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("✅ Trainer created")


In [ ]:
# Train the model
print("🚀 Starting training...")
trainer.train()
print("✅ Training completed!")


## 5. Model Evaluation


In [ ]:
# Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print("Test Results:")
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")

# Generate predictions for visualization
def generate_predictions(model, dataset, num_samples=100):
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for i in range(min(num_samples, len(dataset))):
            sample = dataset[i]
            
            # Prepare input
            inputs = {
                'past_values': sample['past_values'].unsqueeze(0),
                'past_time_features': sample['past_time_features'].unsqueeze(0),
                'static_categorical_features': sample['static_categorical_features'].unsqueeze(0),
                'static_real_features': sample['static_real_features'].unsqueeze(0),
                'future_time_features': sample['future_time_features'].unsqueeze(0)
            }
            
            # Generate prediction
            outputs = model(**inputs)
            prediction = outputs.prediction_outputs
            
            predictions.append(prediction.squeeze().numpy())
            actuals.append(sample['future_values'].numpy())
    
    return np.array(predictions), np.array(actuals)

# Generate predictions
predictions, actuals = generate_predictions(model, test_dataset, num_samples=50)
print(f"Generated {len(predictions)} predictions")


In [ ]:
# Visualize predictions vs actuals
def plot_predictions(predictions, actuals, target_cols, num_targets=4):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    for i in range(min(num_targets, len(target_cols))):
        ax = axes[i]
        
        # Plot first prediction horizon
        pred_values = predictions[:, 0, i]  # First prediction step
        actual_values = actuals[:, 0, i]    # First actual step
        
        ax.scatter(actual_values, pred_values, alpha=0.6)
        ax.plot([actual_values.min(), actual_values.max()], 
                [actual_values.min(), actual_values.max()], 'r--', lw=2)
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        ax.set_title(f'{target_cols[i]}')
        
        # Calculate R²
        r2 = r2_score(actual_values, pred_values)
        ax.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax.transAxes, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

plot_predictions(predictions, actuals, target_cols)


## 6. EBM Model for Blend Effects Explanation


In [ ]:
# Prepare data for EBM
def prepare_ebm_data(dataset, static_cols, target_cols):
    X_static = []
    y_targets = []
    
    for i in range(len(dataset)):
        sample = dataset[i]
        
        # Static features (blend characteristics)
        static_features = sample['static_real_features'].numpy()
        X_static.append(static_features)
        
        # Target values (average across prediction horizon)
        target_values = sample['future_values'].numpy().mean(axis=0)  # Average across time steps
        y_targets.append(target_values)
    
    return np.array(X_static), np.array(y_targets)

# Prepare EBM data
X_static, y_targets = prepare_ebm_data(dataset, static_cols, target_cols)
print(f"EBM data shape: X={X_static.shape}, y={y_targets.shape}")


In [ ]:
# Train EBM models for each target
def train_ebm_models(X_static, y_targets, target_cols, test_size=0.2):
    ebm_models = {}
    ebm_results = {}
    
    for i, target_name in enumerate(target_cols):
        print(f"Training EBM for {target_name}...")
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_static, y_targets[:, i], test_size=test_size, random_state=42
        )
        
        # Train EBM
        ebm = ExplainableBoostingRegressor(
            interactions=10,
            max_bins=256,
            max_interaction_bins=32,
            random_state=42
        )
        
        ebm.fit(X_train, y_train)
        
        # Evaluate
        y_pred = ebm.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        ebm_models[target_name] = ebm
        ebm_results[target_name] = {
            'mse': mse,
            'mae': mae,
            'r2': r2
        }
        
        print(f"  R² = {r2:.3f}, MAE = {mae:.3f}")
    
    return ebm_models, ebm_results

# Train EBM models
ebm_models, ebm_results = train_ebm_models(X_static, y_targets, target_cols)
print("\n✅ EBM models trained successfully!")


In [ ]:
# Visualize EBM feature importance
def plot_ebm_importance(ebm_models, static_cols, target_cols, top_n=10):
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, target_name in enumerate(target_cols[:6]):  # Show first 6 targets
        ax = axes[i]
        ebm = ebm_models[target_name]
        
        # Get feature importance
        importance = ebm.feature_importances_
        
        # Sort by importance
        sorted_idx = np.argsort(importance)[::-1][:top_n]
        
        # Plot
        ax.barh(range(len(sorted_idx)), importance[sorted_idx])
        ax.set_yticks(range(len(sorted_idx)))
        ax.set_yticklabels([static_cols[idx] for idx in sorted_idx])
        ax.set_xlabel('Feature Importance')
        ax.set_title(f'{target_name}\nR² = {ebm_results[target_name]["r2"]:.3f}')
        ax.invert_yaxis()
    
    plt.tight_layout()
    plt.show()

plot_ebm_importance(ebm_models, static_cols, target_cols)


In [ ]:
# Generate EBM explanations for specific samples
def explain_sample(ebm_models, static_cols, target_cols, sample_idx=0):
    from interpret import show
    
    sample_static = X_static[sample_idx]
    sample_targets = y_targets[sample_idx]
    
    print(f"Sample {sample_idx} Explanation:")
    print(f"Static features: {dict(zip(static_cols, sample_static))}")
    print(f"Actual targets: {dict(zip(target_cols, sample_targets))}")
    print("\n" + "="*50)
    
    # Show explanations for each target
    for target_name in target_cols[:3]:  # Show first 3 targets
        ebm = ebm_models[target_name]
        
        # Global explanation
        global_exp = ebm.explain_global()
        print(f"\nGlobal explanation for {target_name}:")
        show(global_exp)
        
        # Local explanation
        local_exp = ebm.explain_local(sample_static.reshape(1, -1), sample_targets[target_cols.index(target_name)])
        print(f"\nLocal explanation for {target_name}:")
        show(local_exp)

# Generate explanations
explain_sample(ebm_models, static_cols, target_cols, sample_idx=0)


## 7. Summary and Results


In [ ]:
# Print summary results
print("="*60)
print("AUTOFORMER + EBM ANALYSIS SUMMARY")
print("="*60)

print("\n📊 Autoformer Performance:")
for key, value in test_results.items():
    if 'eval_' in key:
        print(f"  {key}: {value:.4f}")

print("\n🔍 EBM Model Performance:")
for target_name, results in ebm_results.items():
    print(f"  {target_name}:")
    print(f"    R² = {results['r2']:.3f}")
    print(f"    MAE = {results['mae']:.3f}")
    print(f"    MSE = {results['mse']:.3f}")

print("\n✅ Analysis completed successfully!")
print("\nKey Findings:")
print("1. Autoformer provides multivariate time series forecasting")
print("2. EBM models explain how blend characteristics affect each target")
print("3. Feature importance shows which crude properties matter most")
print("4. Local explanations reveal specific blend effects")
